## 분류분석

### 1. [로지스틱 회귀 분석] 특징 데이터로 유방암 진단하

In [13]:
# Step0: 데이터 준비 및 탐색
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer

# 사이킷런에서 제공하는 데이터셋sklearn.datasets중에서 유방암 진단 데이터셋을 사용하기 위해 load_breast_cancer import
# -> 데이터셋을 로드하여 객체 'b_cancer'를 생성
b_cancer = load_breast_cancer()

# 데이터셋에 대한 설명 확인(description)
b_cancer.DESCR
b_cancer # 이것도 확인해보면, 아래 코드 칠 수 있음

# 데이터셋 객체의 data배열인 b_cancer.data를, 즉 독립 변수 X가 되는 피처를 DataFrame 자료형으로 변환하여 b_cancer_df를생성
b_cancer_df = pd.DataFrame(b_cancer.data, columns = b_cancer.feature_names)

b_cancer_df['diagnosis']= b_cancer.target # 0, 1로 걸렸는지 아닌지 데이터가 있음

# 30개의 피처(독립 변수 X)들과 1개의 종속 변수 이름을 확인 가능
# *diagnosis는 악성이면 1, 양성이면 0의 값으로 유방암 여부에 대한 "이진 분류"의 class로 사용할 종속 변수가 된다
b_cancer_df 

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [17]:
# Step0-2: 로지스틱 회귀 분석에 피처로 사용할 데이터를
    # 평균이 0, 분산이 1이 되는 정규 분포 형태로 맞춘다
    # 이렇게 다 좁혀놔야 제대로 비교가 되겠지

# 사이킷런의 전처리 패키지에 있는 정규 분포 스케일러를 임포트 & 사용할 객체 scaler를 생성
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# 피처로 사용할 데이터b_cancer.data에 대해 정규 분포 스케일링을 수행
    # -> scaler.fit_ transform( )
b_cancer_scaled = scaler.fit_transform(b_cancer.data)
print(b_cancer.data[0]) 
print(b_cancer_scaled[0]) # 정규 분포 스케일링 후에 값이 조정된 것을 확인!!

[1.799e+01 1.038e+01 1.228e+02 1.001e+03 1.184e-01 2.776e-01 3.001e-01
 1.471e-01 2.419e-01 7.871e-02 1.095e+00 9.053e-01 8.589e+00 1.534e+02
 6.399e-03 4.904e-02 5.373e-02 1.587e-02 3.003e-02 6.193e-03 2.538e+01
 1.733e+01 1.846e+02 2.019e+03 1.622e-01 6.656e-01 7.119e-01 2.654e-01
 4.601e-01 1.189e-01]
[ 1.09706398 -2.07333501  1.26993369  0.9843749   1.56846633  3.28351467
  2.65287398  2.53247522  2.21751501  2.25574689  2.48973393 -0.56526506
  2.83303087  2.48757756 -0.21400165  1.31686157  0.72402616  0.66081994
  1.14875667  0.90708308  1.88668963 -1.35929347  2.30360062  2.00123749
  1.30768627  2.61666502  2.10952635  2.29607613  2.75062224  1.93701461]


In [19]:
# (위에서 만든 데이터를 바탕으로)
# Step1: 로지스틱 회귀를 이용하여 분석 모델 구축하기
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# X, Y 설정 -> diagnosis를 Y, 정규 분포로 스케일링한 b_cancer_scaled를 X로 설정
Y = b_cancer_df['diagnosis']
X = b_cancer_scaled

# 훈련용 데이터와 평가용 데이터 분할하기
    # 전체 데이터 샘플 569개를 학습 데이터:평가 데이터=7:3으로 분할test_size=0.3함
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

# (1)로지스틱 회귀 분석 모델 '생성'
    # = 로지스틱 회귀 분석 모델 객체lr_b_cancer를 생성
lr_b_cancer = LogisticRegression()

# (2)로지스틱 회귀 분석 모델 '훈련' & fit=수행=>훈련
lr_b_cancer.fit(X_train, Y_train) # 학습 데이터X_train, Y_train로 모델 학습을 수행fit( )함

# (3)로지스틱 회귀 분석 '평가' - 데이터에 대한 예측 수행 -> 예측 결과 Y_predict 구하기
    #  학습이 끝난 모델에 대해 평가 데이터 X_test를 가지고 예측을 수행 predict( )하여 예측값 Y_predict를 구한다
Y_predict = lr_b_cancer.predict(X_test)
print(Y_predict)


array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1])

In [21]:
# Step2: 생성한 모델의 성능 확인하기
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# 오차 행렬
    # *평가를 위해 7:3으로 분할한 171개의 test 데이터에 대해 '이진' 분류의 성능 평가 기본이 되는 오차 행렬을 구함
    # 실행 결과를 보면 TN이 60개, FP가 3개, FN이 1개, TP가 107개인 오차 행렬이 구해짐
        # array([[ 60,   3],
        #        [  1, 107]], dtype=int64)
confusion_matrix(Y_test, Y_predict)
    

# 성능 평가 지표인 정확도, 정밀도, 재현율, F1 스코어, ROC-AUC 스코어를 구함
acccuracy = accuracy_score(Y_test, Y_predict)
precision = precision_score(Y_test, Y_predict)
recall = recall_score(Y_test, Y_predict)
f1 = f1_score(Y_test, Y_predict)
roc_auc = roc_auc_score(Y_test, Y_predict)

print('정확도: {0:.3f}, 정밀도: {1:.3f}, 재현율: {2:.3f}, F1: {3:.3f}'.format(acccuracy,precision,recall,f1))
print('ROC_AUC: {0:.3f}'.format(roc_auc))

정확도: 0.977, 정밀도: 0.973, 재현율: 0.991, F1: 0.982
ROC_AUC: 0.972
